In [14]:

import np as np
import pandas as pd

### $\int_{-\inf}^{z} e^{\frac{-u^2}{2}}du = \frac{1-0.05}{2}*\sqrt{2*π}$
(обчислення $z_\gamma$ із лекції 7, 2 ст)

In [15]:
z_gamma_empty = 0.062706777

### Завдання 1

Перевірка однорідності (критерій пустих блоків)

Маємо дві незалежні вибірки: $\overline{X} = (X_1, \cdots, X_n), \overline{Y} = (Y_1, \cdots, Y_n)$
1. Розбиваємо вісь $R$ на напівінтервали $(X_{(i-1)}, X_{(i)}], i=1, \cdot, n+1$
2. Обчислюємо кількість пустих блоків: $k = # \text{пустих блоків}$
3. $Crit = \dfrac{n}{1+\rho} + \sqrt{n} \dfrac{\rho}{(1+\rho)^{3/2}} z_{\gamma}$
4. Як $k < Crit$ обираємо $H_0$, інакше $H_1$


In [86]:
import scipy.stats as sps
NM = [[500, 1000], [5000, 10000]]  #, [50000, 100000]]

xi_empty = []
crit_empty = []
RES_empty = []
for nm in NM:

    n = nm[0]
    m = nm[1]
    ro = m / n

    X = np.random.exponential(1, n)
    Y = np.random.exponential(1, m)

    X = sps.expon(1.0).rvs(size=n)
    Y = sps.expon(1.3).rvs(size=m)
 
    X.sort()
    Y.sort()

    # робимо інтервали [1 коорд, 2 коорд]
    intervals = []
    for i in range(1, len(X + 1)):
        intervals.append([X[i - 1], X[i]])

    # рахуємо кількість попадань у відрізок
    res = np.zeros(len(intervals))
    for i, x in enumerate(intervals):
        for item in Y:
            if intervals[i][0] <= item <= intervals[i][1]: res[i] += 1

    xi_empty_temp = np.count_nonzero(res == 0)
    xi_empty.append(xi_empty_temp)

    crit = n / (1 + ro) + np.math.sqrt(n) * ro * z_gamma_empty / (1 + ro) ** (3 / 2)
    crit_empty.append(crit)
    if xi_empty_temp < crit:
        RES_empty.append("H0")
    else:
        RES_empty.append("False")

data_empty = {"xi_empty (n, m)": xi_empty, f"crit": crit_empty, "Result": RES_empty}
df_empty = pd.DataFrame.from_dict(data_empty, orient='index', columns=[f"n::{item[0]}; m::{item[1]}" for item in NM])


In [87]:
df_empty

,n::500; m::1000,n::5000; m::10000
"xi_empty (n, m)",167,1638
crit,167.206361,1668.373329
Result,H0,H0


## Завдання 2

### Ранжування

In [19]:
rank = lambda data: pd.Series(data).rank(method="min").to_list()
gamma = 0.05
z_gamma = 0.0627

### Критерій перевірки гіпотези незалежності $H_0$ / Критерій Спірмана

1) Маємо вибірку $(\overline{X}, \overline{Y}) = { (X_1, Y_1), \cdots (X_n, Y_n) }$
2) Обчислюємо $R_i$ -- ранг $X_i$ серед елементів $X_1, \cdots, X_n$
3) Обчислюємо $S_i$ -- ранг $Y_i$ серед елементів $Y_1, \cdots, Y_n$
4) Маємо пари $(\overline{R}, \overline{S}) = { (R_1, S_1), \cdots (R_n, S_n) }$
5) Сортуємо за $R_i \Rightarrow (1, V_1), \cdots (n, V_n)$
6) Рахуємо статистику $\rho_n(\bar X, \bar Y) =  1 - \frac{6}{n (n^{2} - 1)} \cdot \sum\limits_{i \in [1,n]} (R_{i} - S_{i})^{2}$ (коеф кореляції двох множин рангів)
7) Порівнюємо $\rho_n \bigvee \dfrac{z_\gamma}{\sqrt{n}}$, як $<$ приймаємо $H_0$, інакше $H_1$

In [79]:
def SpearMan(V):
    # тут V = [(R, S), ()...]
    n = len(V)
    return 1 - 6 * np.sum(pow(V[:, 0] - V[:, 1], 2)) / (n * (pow(n, 2) - 1))
    ...

### пункт а) $Y_{i} = \xi_{i} \cdot \eta_{i}$

In [89]:
from scipy import stats as sc
import math

gamma = 0.05
N_arr = [500, 5000]


# критерій Спірмана
z = sc.norm.ppf(1 - gamma/2)

def Spearman(X,Y):
    n = len(X)
    X_copy = X.copy()
    X_copy.sort()
    Y_copy = Y.copy()
    Y_copy.sort()
    R = np.array([np.where(X_copy == X[i])[0][0] for i in range(n)])
    S = np.array([np.where(Y_copy == Y[i])[0][0] for i in range(n)])
    # згенерували рангові масиви
    po = 1 - 6 / (n * (n ** 2 - 1)) * sum([(R[i] - S[i]) ** 2 for i in range(n)])
    

    limit = z/math.sqrt(n)
    po = abs(po)
    if po < limit:
        print('H0')
        print(f'Spearman = {po}, crit = {limit}')
    else:
        print('False')
        print(f'Spearman = {po}, crit = {limit}')

for n in N_arr:

    X = np.random.uniform(low = 0.0, high = 1.0, size = n)
 
    Eta = np.random.uniform(low = -1.0, high = 1.0, size = n)
 
    Y = []
    for i in range(len(X)):
        Y.append(X[i]*Eta[i])
    print(f"n:{n}\n {Spearman(X,Y)}")

    

H0
Spearman = 0.026814539258157022, crit = 0.08765225405765815
n:500
 None
H0
Spearman = 0.013994613967784564, crit = 0.027718076486993554
n:5000
 None


### пункт б) $Y_{i} = \xi_{i} + \eta_{i}$

In [51]:
N_arr = [500, 5000, 50000]

z_gamma_spearman = [z_gamma / np.math.sqrt(n) for n in N_arr]

RHO = []
Hyp_Spearman = []
for i, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_eta = np.random.uniform(-1, 1, n)
    X = data_xi
    Y = data_xi + data_eta

    print(f"cov: {np.cov(X, Y)}")

    res = np.array([X, Y])
    # паруємо
    res = np.transpose(res)
    # сортуємо по Х
    # res = np.argsort(res, 0)

    # обчислюємо статистику
    test = abs(SpearMan(res))
    RHO.append(test)
    if test < z_gamma_spearman[i]:
        Hyp_Spearman.append("H0")
    else:
        Hyp_Spearman.append("False")

data_Spearman = {"Spearman statistic": RHO, f"z_gamma": z_gamma_spearman, "Result": Hyp_Spearman}
df_Spearman = pd.DataFrame.from_dict(data_Spearman, orient='index', columns=N_arr)

df_Spearman

cov: [[0.08552954 0.09433286]
 [0.09433286 0.42601294]]
cov: [[0.08404904 0.07661595]
 [0.07661595 0.4023396 ]]
cov: [[0.08348962 0.08381507]
 [0.08381507 0.41530049]]


,500,5000,50000
Spearman statistic,0.999992,1.0,1.0
z_gamma,0.002804,0.000887,0.00028
Result,False,False,False


### Критерій перевірки гіпотези незалежності $H_0$ / критерій Кендала
1) Маємо вибірку $(\overline{X}, \overline{Y}) = { (X_1, Y_1), \cdots (X_n, Y_n) }$
2) Обчислюємо $R_i$ -- ранг $X_i$ серед елементів $X_1, \cdots, X_n$
3) Обчислюємо $S_i$ -- ранг $Y_i$ серед елементів $Y_1, \cdots, Y_n$
4) Маємо пари $(\overline{R}, \overline{S}) = { (R_1, S_1), \cdots (R_n, S_n) }$
5) Сортуємо за $R_i \Rightarrow (1, V_1), \cdots (n, V_n)$
6) Обчислюємо $N$: #пари індексів $(i, j), i < j$ для яких $V_i < V_j$
7) Обчислюємо критичну область $\tau = \frac{4N}{n(n-1)} - 1$


### для обчислення гіпотези використовується

### пункт а) $Y_{i} = \xi_{i} \cdot \eta_{i}$

In [52]:
N_arr = [500, 5000, 50000]

z_gamma_kendal = [z_gamma / np.math.sqrt(n) / 3 * 2 for n in N_arr]
tau = []
Hyp_Kendal = []
for tmp, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = data_xi
    Y = data_xi * data_nu

    res = np.array([X, Y], np.int64)
    # паруємо
    res = np.transpose(res)
    # сортуємо по Х
    res = np.argsort(res, 0)

    N_1 = 0
    for i in res[1:-1, 0]:
        if res[i, 1] > res[i - 1, 1]: N_1 += np.sign(res[i, 1] - res[i - 1, 1])

    N = n * (n - 1) / 2 - N_1
    res = 4 * N / (n * (n - 1)) - 1
    tau.append(res)
    if res < z_gamma_kendal[tmp]:
        Hyp_Kendal.append("H0")
    else:
        Hyp_Kendal.append("false")

data_Kendal = {"Kendal statistic": tau, f"z_gamma": z_gamma_kendal, "Result": Hyp_Kendal}
df_Kendal = pd.DataFrame.from_dict(data_Kendal, orient='index', columns=N_arr)

df_Kendal

,500,5000,50000
Kendal statistic,0.999102,0.999823,0.999934
z_gamma,0.001869,0.000591,0.000187
Result,false,false,false


### пункт б) $Y_{i} = \xi_{i} + \eta_{i}$

In [53]:
N_arr = [500, 5000, 50000]

z_gamma_kendal = [z_gamma / np.math.sqrt(n) / 3 * 2 for n in N_arr]
tau = []
Hyp_Kendal = []
for tmp, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = data_xi
    Y = data_xi + data_nu

    res = np.array([X, Y], np.int64)
    # паруємо
    res = np.transpose(res)
    # сортуємо по Х
    res = np.argsort(res, 0)

    N_1 = 0
    for i in res[1:-1, 0]:
        if res[i, 1] > res[i - 1, 1]: N_1 += np.sign(res[i, 1] - res[i - 1, 1])

    N = n * (n - 1) / 2 - N_1
    res = 4 * N / (n * (n - 1)) - 1
    tau.append(res)
    if res < z_gamma_kendal[tmp]:
        Hyp_Kendal.append("H0")
    else:
        Hyp_Kendal.append("false")

data_Kendal = {"Kendal statistic": tau, f"z_gamma": z_gamma_kendal, "Result": Hyp_Kendal}
df_Kendal = pd.DataFrame.from_dict(data_Kendal, orient='index', columns=N_arr)

df_Kendal

,500,5000,50000
Kendal statistic,0.99782,0.999413,0.999943
z_gamma,0.001869,0.000591,0.000187
Result,false,false,false


### Завдання 3
#### Критерій інверсій

У даному методі будемо перевіряти гіпотезу випадковості (повного хаосу), що означає:
$$H_0: F_{\overline{x}}(u_1, \ldots, u_n) = F_{\xi}(u_1) \cdot F_{\xi}(u_2) \cdot \ldots \cdot F_{\xi}(u_n)$$

**Алгоритм обчислення:**
1. Згенерувати $n$ спостережень
2. Обчислюємо "хаос" $k$ у варіаційному ряді,
3. Обчислюємо критичну область $\dfrac{z_{\gamma} n \sqrt{n}}{6}$
4. Рахуємо статистику $S_n(\overline{X}) = \Big| k - \dfrac{n(n-1)}{4}  \Big|$
5. Порівнюємо $ \Big| k - \dfrac{n(n-1)}{4}  \Big| \bigvee  \dfrac{z_{\gamma} n \sqrt{n}}{6}$
як $<$, приймаємо гіпотезу $H_0$, інакше -- ні

In [25]:
N_arr = [500, 5000]  #, 50000]
z_gamma_inv = [z_gamma / 6 * n ** (3 / 2) for n in N_arr]
LAMBDA = []
RES_inv = []


def genX(n):
    X = []
    ksi = np.random.uniform(-1, 1, n)
    for j in range(n):
        x = np.sum(ksi[:j + 1]) / (j + 1)
        X.append(x)
    return X


for tmp, n in enumerate(N_arr):
    X = genX(n)
    k = 0

    for x in range(1, len(X) - 1):
        for j in range(x, len(X)):
            i = j - 1
            if (X[j] < X[i]) and (i < j): k += 1

    print(f"k_{tmp}: {k}")

    statistic = (abs(k - n * (n - 1) / 4))
    LAMBDA.append(statistic)
    if statistic < z_gamma_inv[tmp]:
        RES_inv.append("H0")
    else:
        RES_inv.append("false")

data_inv = {"LAMBDA": LAMBDA, f"z_gamma": z_gamma_inv, "Result": RES_inv}
df_inv = pd.DataFrame.from_dict(data_inv, orient='index', columns=N_arr)

df_inv

k_0: 53259
k_1: 6181302


,500,5000
LAMBDA,9116.0,67448.0
z_gamma,116.834552,3694.632932
Result,false,false
